In [1]:
from utils import seed_torch
from train import main
import train_configs
import models

/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1802: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1828: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,
/home/kaor/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


In [2]:
config1 = train_configs.config16v6samplerAugMixB3.config
seed_torch(config1.seed)

In [4]:
import torch
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from models.eff import get_model
from runner import *
from post_processing import *
from datasets import *
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
     data = bz2.BZ2File(file, 'rb')
     data = cPickle.load(data)
     return data

In [5]:
train = pd.read_csv(config1.kfold_csv)
test = pd.read_csv('../../whale/sample_submission.csv')

In [6]:
import glob
names = glob.glob("/sise/liorrk-group/OrDanOfir/output/*fold0*.pbz2")
names = names[:-2]
names

['/sise/liorrk-group/OrDanOfir/output/exp16v6AugmixB4_fold0.pbz2',
 '/sise/liorrk-group/OrDanOfir/output/exp16v6AugmixB3_fold0.pbz2',
 '/sise/liorrk-group/OrDanOfir/output/exp16v4sampler_fold0.pbz2',
 '/sise/liorrk-group/OrDanOfir/output/exp16v6Augmix_fold0.pbz2',
 '/sise/liorrk-group/OrDanOfir/output/exp16v6sampler_fold0.pbz2',
 '/sise/liorrk-group/OrDanOfir/output/exp21b6_fold0.pbz2']

In [7]:
EMB_test = []
EMB_val = []
for exp in tqdm(names):
    print(exp)
    data = decompress_pickle(exp)
    EMB_test.append(data['emb_test'])
    EMB_val.append(data['emb_val'])
    targets_val = data['targets_val']
    targets_filename = data['targets_filename']
emb_test = np.concatenate(EMB_test,-1)
emb_val = np.concatenate(EMB_val,-1)

  0%|          | 0/6 [00:00<?, ?it/s]

/sise/liorrk-group/OrDanOfir/output/exp16v6AugmixB4_fold0.pbz2


 17%|█▋        | 1/6 [00:09<00:48,  9.70s/it]

/sise/liorrk-group/OrDanOfir/output/exp16v6AugmixB3_fold0.pbz2


 33%|███▎      | 2/6 [00:19<00:38,  9.70s/it]

...

 67%|██████▋   | 4/6 [00:38<00:19,  9.56s/it]

/sise/liorrk-group/OrDanOfir/output/exp16v6sampler_fold0.pbz2


 83%|████████▎ | 5/6 [00:47<00:09,  9.52s/it]

/sise/liorrk-group/OrDanOfir/output/exp21b6_fold0.pbz2


100%|██████████| 6/6 [00:57<00:00,  9.54s/it]


In [8]:
def map_per_image(label, predictions):
    indexes = np.unique(predictions, return_index=True)[1]
    predictions = [predictions[index] for index in sorted(indexes)]
    t = np.where(np.array(predictions[:5]).astype(int)==label)[0]
    if len(t)>0:
        return 1 / (t[0] + 1)
    else:
        return 0.0

In [9]:
train = pd.read_csv(config1.kfold_csv)
test = pd.read_csv('../../whale/sample_submission.csv')
df = pd.concat([train,pd.DataFrame(emb_val)],1)
df_train = df[(df.fold!=0) & (df.fold!=-1)]
df_val = df[(df.fold==0) | (df.fold==-1)]
df_val.reset_index(drop=True,inplace=True)
df_train.reset_index(drop=True,inplace=True)
targets = df_train['classes'].values

/home/kaor/.conda/envs/openmmlab/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
vals_blend = []
labels_blend = []
inds_blend = []
for i in range(len(EMB_test)):    
    vals, inds = get_topk_cossim_sub(df_val.values[:,8:].astype(float)[:,i*512:(1+i)*512], df_train.values[:,8:].astype(float)[:,i*512:(1+i)*512], k=1500)
    vals = vals.data.cpu().numpy()
    inds = inds.data.cpu().numpy()
    labels = np.concatenate([targets[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)
    vals_blend.append(vals)
    labels_blend.append(labels)
    inds_blend.append(inds)

100%|██████████| 276/276 [00:00<00:00, 943.93it/s]


In [11]:
len(vals_blend[0])

17630

In [12]:
rank = np.zeros([len(df_val),16000])
score = np.zeros([len(df_val),16000])
std_rank = np.zeros([len(df_val),16000])
std_score = np.zeros([len(df_val),16000])

In [13]:
w = [1]*len(labels_blend)
for row in tqdm(range(len(labels_blend))):
    for i in range(len(labels_blend[row])):
        for j,l in enumerate(labels_blend[row][i][0:50]):
            rank[i,l] += 1/(j+1)
            score[i,l]+=vals_blend[row][i][j]

100%|██████████| 6/6 [00:07<00:00,  1.30s/it]


In [14]:
i=0
rank[i].argmax(),rank[i].max(),score[i].max()

(4733, 4.0222386222023925, 13.00985249876976)

In [15]:
rank.argmax(1).shape

(17630,)

In [73]:
df_val2 = df_val[df_val.columns[0:8]].copy()

In [74]:
rank2 = rank.argsort(1)
rank_max = np.sort(rank,axis=1)
score_max = np.sort(score,axis=1)

In [75]:
vals_blend = np.array(vals_blend)
vals_blend[:,:,0].std(0)

array([0.04920809, 0.05721493, 0.04086269, ..., 0.04066165, 0.04967054,
       0.02944606], dtype=float32)

In [76]:
for row in range(5):
    df_val2[f'pred{row}'] = rank2[:,-(row+1)]
    df_val2[f'rank{row}'] = rank_max[:,-(row+1)]
    df_val2[f'score{row}'] = score_max[:,-(row+1)]
    df_val2[f'vals_blend{row}'] = vals_blend[:,:,row].std(0)
    df_val2[f'vals_mean{row}'] = vals_blend[:,:,row].mean(0)
df_val2['score_std'] = score.std(1)
df_val2['rank_std'] = rank.std(1)
df_val2['score_mean'] = score.mean(1)
df_val2['rank_mean'] = rank.mean(1)

In [77]:
6746/(df_val2.fold==0).sum(),df_val2[df_val2.pred0==df_val2.classes]['rank0'].min()

(0.8057811753463927, 1.8903883199005154)

In [78]:
df_val3 = df_val2.sample(frac=1.0)
len(df_val3)

17630

In [81]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
cl = [f'pred{row}' for row in range(5)]
cl += [f'rank{row}' for row in range(5)]
cl += [f'score{row}' for row in range(5)]
cl += [f'vals_blend{row}' for row in range(5)]
cl += [f'vals_mean{row}' for row in range(5)]
cl +=['score_std','rank_std','score_mean','rank_mean']
reg = RandomForestClassifier(max_depth=5, random_state=0).fit(df_val3[cl].values[:10000], df_val3['fold'].values[:10000])
reg.score(df_val3[cl].values[10000:], df_val3['fold'].values[10000:])

0.8475753604193971

In [82]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
X_train = df_val3[cl].values[:10000]
y_train = df_val3['fold'].values[:10000]+1
X_test = df_val3[cl].values[10000:]
y_test = df_val3['fold'].values[10000:]+1
clf = lgb.LGBMClassifier(learning_rate=0.07,max_depth=10,random_state=42)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
accuracy

0.8623853211009175